Indlæs pakker:

In [ ]:
#Indlæs nødvendige biblioteker
import os
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset

#Data augmentation
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm

import torchvision.transforms as T
import random
import numpy as np
import cv2
from torchvision.transforms import RandAugment

#Split data
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

#ResNet-18:
import torch.nn as nn
import torchvision
from torchvision.models import ResNet18_Weights

#Træning og validation
from typing import Tuple
from typing import Dict, List

from torch.utils.tensorboard import SummaryWriter

#Plotting
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

#Grad-cam
import torch.nn.functional as F
from torchvision import models, transforms

In [ ]:
class CFG:
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    NUM_DEVICES = torch.cuda.device_count()
    NUM_WORKERS = os.cpu_count()
    NUM_CLASSES = 30
    EPOCHS = 30
    BATCH_SIZE = (
        256 if torch.cuda.device_count() < 2 
        else (256 * torch.cuda.device_count())
    )
    TEST_SIZE = 0.15
    LR = 0.001
    LR_STEP_SIZE = 10
    LR_GAMMA = 0.1
    APPLY_SHUFFLE = True
    SEED = 768
    HEIGHT = 224
    WIDTH = 224
    CHANNELS = 3
    IMAGE_SIZE = (224, 224, 3)

Indlæs datasæt ved hjælp fra csv fil.

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row["file_name"]).convert("RGB")
        label = torch.tensor(row["label_idx"]).long()
        image_path = row["file_name"]
        if self.transform:
            image = self.transform(image)

        return image, label, image_path

def load_dataset(csv_path: str, image_dir: str) -> pd.DataFrame:
    """
    Loads the filtered dataframe, appends full image paths, encodes labels,
    and checks for missing files.

    Returns:
        df: Pandas DataFrame with encoded labels and image paths
        idx_to_label: Dictionary mapping encoded label indices back to strings
    """
    df = pd.read_csv(csv_path)

    df["file_name"] = df["file_name"].astype(str) + ".png"
    df["file_name"] = df["file_name"].apply(lambda x: os.path.join(image_dir, x))

    missing = df[~df["file_name"].apply(os.path.exists)]
    if not missing.empty:
        print(f"[Warning] Missing images: {len(missing)}")

    le = LabelEncoder()
    df["label_idx"] = le.fit_transform(df["label"])

    idx_to_label = dict(enumerate(le.classes_))
    return df, idx_to_label

In [ ]:
csv_path = ""
image_dir = ""
df, idx_to_label = load_dataset(csv_path, image_dir)

Udregn gennemsnit og standard afvigelse

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = datasets.CIFAR10(
    root="", transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=True)


def get_mean_std(loader):
    # var[X] = E[X**2] - E[X]**2
    channels_sum, channels_sqrd_sum, num_batches = 0, 0, 0

    for data, _ in tqdm(loader):
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_sqrd_sum += torch.mean(data**2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_sqrd_sum / num_batches - mean**2) ** 0.5

    return mean, std


mean, std = get_mean_std(train_loader)
print(mean)
print(std)

Data Augmentation

In [ ]:
class CannyEdgeTransform:
    def __init__(self, p=0.3, threshold1=100, threshold2=200):
        self.p = p
        self.threshold1 = threshold1
        self.threshold2 = threshold2

    def __call__(self, img):
        if random.random() < self.p:
            img_np = np.array(img)
            gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
            edges = cv2.Canny(gray, self.threshold1, self.threshold2)
            edges_rgb = np.stack([edges]*3, axis=-1)
            img = Image.fromarray(edges_rgb)
        return img


def random_transforms():
    """
    Returns train and val transforms with a custom random augmentation policy.
    """

    train_transforms = T.Compose([
        T.Resize((CFG.HEIGHT, CFG.WIDTH)),
        RandAugment(num_ops=3, magnitude=12),
        CannyEdgeTransform(p=0.3),
        T.ToTensor(),
        T.Normalize(mean=[0.4914, 0.4821, 0.4465], std=[0.2470, 0.2435, 0.2615]),
    ])


    val_transforms = T.Compose([
        T.Resize((CFG.HEIGHT, CFG.WIDTH)),
        T.ToTensor(),
        T.Normalize(mean=[0.4914, 0.4821, 0.4465], std=[0.2470, 0.2435, 0.2615]),
    ])

    return train_transforms, val_transforms

Split data i træning, validering og test

In [ ]:
def get_dataloaders(df, train_transforms, val_transforms):
    """
    Returns train, validation, and test datasets and dataloaders.
    """

    # First split: Train + Temp (val + test)
    train_df, temp_df = train_test_split(
        df,
        test_size=CFG.TEST_SIZE,
        stratify=df["label_idx"],
        random_state=CFG.SEED
    )

    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.4,
        stratify=temp_df["label_idx"],
        random_state=CFG.SEED
    )

    train_dataset = ImageDataset(train_df, transform=train_transforms)
    val_dataset = ImageDataset(val_df, transform=val_transforms)
    test_dataset = ImageDataset(test_df, transform=val_transforms)

    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.BATCH_SIZE,
        shuffle=CFG.APPLY_SHUFFLE,
        num_workers=CFG.NUM_WORKERS,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=CFG.BATCH_SIZE,
        shuffle=False,
        num_workers=CFG.NUM_WORKERS
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=CFG.BATCH_SIZE,
        shuffle=False,
        num_workers=CFG.NUM_WORKERS
    )

    return train_dataset, val_dataset, test_dataset, train_loader, val_loader, test_loader

In [ ]:
train_transforms, val_transforms = random_transforms()

train_dataset, val_dataset, test_dataset, train_loader, val_loader, test_loader = get_dataloaders(
    df, train_transforms, val_transforms
)

ResNet-18 - lag 2 og 3 bliver 'unfrozen'

In [ ]:
def build_model(device: torch.device, num_classes: int = CFG.NUM_CLASSES) -> nn.Module:
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)

    weights = ResNet18_Weights.DEFAULT
    model = torchvision.models.resnet18(weights=weights).to(device)

    # Freeze all layers
    for param in model.parameters():
        param.requires_grad = False

    unfreeze_layers = [model.layer2, model.layer3]
    for layer in unfreeze_layers:
        for param in layer.parameters():
            param.requires_grad = True

    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(256, num_classes),
    ).to(device)

    return model

In [ ]:
cnn = build_model(device=CFG.DEVICE)

# View model summary
summary(
    model=cnn, 
    input_size=(CFG.BATCH_SIZE, CFG.CHANNELS, CFG.WIDTH, CFG.HEIGHT),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

In [ ]:
# Define Loss Function
loss_fn = nn.CrossEntropyLoss(
    label_smoothing=0.1
)

 #Define Optimizer
optimizer = torch.optim.Adam(
    cnn.parameters(),
    lr=CFG.LR)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=CFG.LR_STEP_SIZE, gamma=CFG.LR_GAMMA)

Definer træningsscript for hver epoch. Loss, Accuracy og top-3 accuracy

In [ ]:
def execute_epoch(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    device: torch.device
) -> Tuple[float, float, float]:
    """
    Executes a single training epoch and returns average loss, top-1 and top-5 accuracy.
    """

    model.train()
    train_loss, top1_acc, top3_acc = 0.0, 0.0, 0.0

    for X, y, _ in tqdm(dataloader):
        X, y = X.to(device), y.to(device)

        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        top1 = torch.argmax(y_pred, dim=1)
        top1_acc += (top1 == y).sum().item() / y.size(0)

        top3_preds = torch.topk(y_pred, k=3, dim=1).indices
        match_top3 = top3_preds.eq(y.view(-1, 1))  # shape: [batch, 5]
        top3_acc += match_top3.any(dim=1).float().sum().item() / y.size(0)

    num_batches = len(dataloader)
    train_loss /= num_batches
    top1_acc /= num_batches
    top3_acc /= num_batches

    return train_loss, top1_acc, top3_acc

def evaluate(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: torch.device
) -> Tuple[float, float, float]:
    """
    Evaluates model performance on a validation/test set.

    Returns:
        avg_loss, top1_accuracy, top3_accuracy
    """

    model.eval()
    eval_loss, top1_acc, top3_acc = 0.0, 0.0, 0.0

    with torch.inference_mode():
        for X, y, _ in dataloader:
            X, y = X.to(device), y.to(device)

            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            eval_loss += loss.item()

            top1 = torch.argmax(y_pred, dim=1)
            top1_acc += (top1 == y).sum().item() / y.size(0)


            top3_preds = torch.topk(y_pred, k=3, dim=1).indices
            match_top3 = top3_preds.eq(y.view(-1, 1))
            top3_acc += match_top3.any(dim=1).float().sum().item() / y.size(0)


    num_batches = len(dataloader)
    eval_loss /= num_batches
    top1_acc /= num_batches
    top3_acc /= num_batches

    return eval_loss, top1_acc, top3_acc

def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    eval_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs: int,
    device: torch.device,
    writer: SummaryWriter,
    save_path="",
    scheduler: torch.optim.lr_scheduler._LRScheduler = None
) -> Dict[str, List]:
    """
    Trains and evaluates a model over multiple epochs and logs top-1 and top-3 accuracy.
    """

    session = {
        'loss': [],
        'accuracy': [],
        'top3_accuracy': [],
        'eval_loss': [],
        'eval_accuracy': [],
        'eval_top3_accuracy': []
    }

    best_eval_loss = float("inf")

    for epoch in range(epochs):
        print(f'\nEpoch {epoch + 1}/{epochs}')

        # Training
        train_loss, train_top1, train_top3 = execute_epoch(
            model, train_dataloader, optimizer, loss_fn, device
        )

        # Evaluation
        eval_loss, eval_top1, eval_top3 = evaluate(
            model, eval_dataloader, loss_fn, device
        )

        # ✅ Save best model based on validation loss
        if eval_loss < best_eval_loss:
            best_eval_loss = eval_loss
            torch.save(model.state_dict(), os.path.join(save_path, "best_model.pth"))
            print(f"✅ Best model saved at epoch {epoch + 1} with eval_loss = {best_eval_loss:.4f}")


        current_lr = optimizer.param_groups[0]["lr"]
        print(f"Learning rate for epoch {epoch+1}: {current_lr:.6f}")

        # Log metrics to TensorBoard ✅
        writer.add_scalar("Loss/Train", train_loss, epoch)
        writer.add_scalar("Loss/Val", eval_loss, epoch)
        writer.add_scalar("Accuracy/Train_top1", train_top1, epoch)
        writer.add_scalar("Accuracy/Val_top1", eval_top1, epoch)
        writer.add_scalar("Accuracy/Train_top3", train_top3, epoch)
        writer.add_scalar("Accuracy/Val_top3", eval_top3, epoch)

        # Log current LR
        if scheduler is not None:
            current_lr = scheduler.get_last_lr()[0]
            writer.add_scalar("Learning_Rate", current_lr, epoch)
            scheduler.step()

        # Print logs
        print(
            f'loss: {train_loss:.4f} - top1: {train_top1:.4f} - top3: {train_top3:.4f} '
            f'- eval_loss: {eval_loss:.4f} - eval_top1: {eval_top1:.4f} - eval_top3: {eval_top3:.4f}'
        )

        # Save to session log
        session['loss'].append(train_loss)
        session['accuracy'].append(train_top1)
        session['top3_accuracy'].append(train_top3)
        session['eval_loss'].append(eval_loss)
        session['eval_accuracy'].append(eval_top1)
        session['eval_top3_accuracy'].append(eval_top3)

    return session

In [ ]:
writer = SummaryWriter(log_dir="runs/exp2_final")

session = train(
    model=cnn,
    train_dataloader=train_loader,
    eval_dataloader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=CFG.EPOCHS,
    device=CFG.DEVICE,
    writer=writer,
    scheduler=scheduler
)

writer.close()

In [ ]:
session_history_df = pd.DataFrame(session)
session_history_df

Træning og valideringskurve

In [ ]:
def plot_training_curves(history):
    
    loss = np.array(history['loss'])
    val_loss = np.array(history['eval_loss'])

    accuracy = np.array(history['accuracy'])
    val_accuracy = np.array(history['eval_accuracy'])

    epochs = range(len(history['loss']))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # Plot loss
    ax1.plot(epochs, loss, label='training_loss', marker='o')
    ax1.plot(epochs, val_loss, label='eval_loss', marker='o')
    
    ax1.fill_between(epochs, loss, val_loss, where=(loss > val_loss), color='C0', alpha=0.3, interpolate=True)
    ax1.fill_between(epochs, loss, val_loss, where=(loss < val_loss), color='C1', alpha=0.3, interpolate=True)

    ax1.set_title('Loss (Lower Means Better)', fontsize=16)
    ax1.set_xlabel('Epochs', fontsize=12)
    ax1.legend()

    # Plot accuracy
    ax2.plot(epochs, accuracy, label='training_accuracy', marker='o')
    ax2.plot(epochs, val_accuracy, label='eval_accuracy', marker='o')
    
    ax2.fill_between(epochs, accuracy, val_accuracy, where=(accuracy > val_accuracy), color='C0', alpha=0.3, interpolate=True)
    ax2.fill_between(epochs, accuracy, val_accuracy, where=(accuracy < val_accuracy), color='C1', alpha=0.3, interpolate=True)

    ax2.set_title('Accuracy (Higher Means Better)', fontsize=16)
    ax2.set_xlabel('Epochs', fontsize=12)
    ax2.legend();
    
    sns.despine();
    
    return

In [ ]:
# Plot EfficientNet session training history 
plot_training_curves(session)

Classification report

In [ ]:
def get_predictions(model, dataloader=val_loader, device=CFG.DEVICE):
    model.eval()
    all_preds = []
    all_probs = []
    all_labels = []
    all_paths = []

    with torch.inference_mode():
        for images, labels, paths in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_paths.extend(paths)

    return np.array(all_labels), np.array(all_preds), np.array(all_probs), all_paths

In [ ]:
y_true, y_pred, y_probs, file_paths = get_predictions(cnn, val_loader, CFG.DEVICE)

# Classification Report
print("📋 Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=list(idx_to_label.values()), labels=list(idx_to_label.keys())))

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=idx_to_label.values(), yticklabels=idx_to_label.values(), cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

Hvilke billeder gætter modellen meget forkert.

In [ ]:
import os
import numpy as np

def print_confidently_wrong(y_true, y_pred, y_probs, file_paths, idx_to_label, top_n=10):
    """
    Prints the top N confidently wrong predictions based on prediction probability.
    """

    wrong_idx = np.where(y_pred != y_true)[0]

    if len(wrong_idx) == 0:
        print("✅ No misclassifications found.")
        return

    confidences = [y_probs[i][y_pred[i]] for i in wrong_idx]

    sorted_indices = np.argsort(confidences)[-top_n:][::-1]
    top_wrong_idx = [wrong_idx[i] for i in sorted_indices]

    print(f"\n🔍 Top {len(top_wrong_idx)} confident misclassifications:\n")

    for i in top_wrong_idx:
        true_label = idx_to_label[y_true[i]]
        pred_label = idx_to_label[y_pred[i]]
        confidence = y_probs[i][y_pred[i]]
        image_path = file_paths[i]
        print(f"🖼️ {os.path.basename(image_path)} | ❌ Pred: {pred_label} ({confidence:.2f}) | ✅ True: {true_label}")

Specifikke typer som bliver gættet forkert

In [ ]:
def show_specific_misclassifications(y_true, y_pred, file_paths, idx_to_label, target_true="MEL1", target_pred="MEL2", max_images=12):
    """
    Displays images where the predicted label is 'target_pred' but the true label is 'target_true'.

    Args:
        y_true: array of ground truth label indices
        y_pred: array of predicted label indices
        file_paths: list of image file paths
        idx_to_label: dictionary mapping label indices to label strings
        target_true: class name that should be true
        target_pred: class name that was wrongly predicted
        max_images: maximum number of images to display
    """

    # Convert labels to class names
    true_labels = [idx_to_label[i] for i in y_true]
    pred_labels = [idx_to_label[i] for i in y_pred]

    # Filter indices
    match_indices = [
        i for i, (true, pred) in enumerate(zip(true_labels, pred_labels))
        if true == target_true and pred == target_pred
    ]

    if not match_indices:
        print(f"No misclassifications found where predicted = {target_pred} but true = {target_true}.")
        return

    # Plot
    fig, axes = plt.subplots(1, min(len(match_indices), max_images), figsize=(15, 5))
    if len(match_indices) == 1:
        axes = [axes]

    for ax, idx in zip(axes, match_indices[:max_images]):
        img = Image.open(file_paths[idx])
        ax.imshow(img)
        ax.set_title(f"Pred: {target_pred}\nTrue: {target_true}")
        ax.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
show_specific_misclassifications(
    y_true=y_true,
    y_pred=y_pred,
    file_paths=file_paths,
    idx_to_label=idx_to_label,
    target_true="BC",
    target_pred="BC K.M.skab",
    max_images=16
)

GRAD-Cam

In [ ]:
# ----- Load model -----
model = build_model(device="cuda", num_classes=30)
model.load_state_dict(torch.load(""))
model.eval()

target_layer = model.layer4

gradients = []

def save_gradient(module, grad_input, grad_output):
    gradients.append(grad_output[0])

target_layer.register_backward_hook(save_gradient)

def preprocess(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4821, 0.4465], std=[0.2470, 0.2435, 0.2615])
    ])
    img = Image.open(image_path).convert("RGB")
    return transform(img).unsqueeze(0), img

# ----- Grad-CAM logic -----
def generate_cam(model, image_tensor, class_idx):
    activations = []

    def forward_hook(module, input, output):
        activations.append(output)

    hook = target_layer.register_forward_hook(forward_hook)

    output = model(image_tensor)
    pred_class = output.argmax(dim=1).item() if class_idx is None else class_idx

    model.zero_grad()
    class_score = output[0, pred_class]
    class_score.backward()

    act = activations[0].squeeze(0).detach().cpu().numpy()
    grad = gradients[0].squeeze(0).detach().cpu().numpy()

    weights = np.mean(grad, axis=(1, 2))
    cam = np.zeros(act.shape[1:], dtype=np.float32)

    for i, w in enumerate(weights):
        cam += w * act[i]

    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (224, 224))
    cam -= np.min(cam)
    cam /= (np.max(cam) + 1e-8)

    hook.remove()
    return cam

# ----- Plot result -----
def show_cam_on_image(original_img, cam):
    img = np.array(original_img.resize((224, 224))) / 255.0
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = heatmap[..., ::-1] / 255.0
    superimposed_img = 0.5 * heatmap + 0.5 * img
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.title("Grad-CAM")
    plt.show()

# ----- Run -----
image_tensor, original_img = preprocess("")
image_tensor = image_tensor.to("cuda")
cam = generate_cam(model, image_tensor, class_idx=None)
show_cam_on_image(original_img, cam)

Predictions på testsæt

In [ ]:
all_preds, all_labels, all_paths = [], [], []

with torch.no_grad():
    for images, labels, paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(torch.softmax(outputs, dim=1), dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_paths.extend(paths)

results = pd.DataFrame({
    "file_name": all_paths,
    "true_label": all_labels,
    "pred_label": all_preds
})
results["true_label_name"] = results["true_label"].map(idx_to_label)
results["pred_label_name"] = results["pred_label"].map(idx_to_label)

results.to_csv("test_predictions.csv", index=False)
print("✅ Predictions saved to test_predictions.csv")

In [ ]:
accuracy = (results["true_label"] == results["pred_label"]).mean()
print(f"Test set accuracy: {accuracy:.2f}")